In [1]:
import time
ini_time=time.time()

import datetime

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

%run 'prev_comex_ncm.ipynb'

# Define parametros
cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']
l=20

comex_ncm=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])

# Filtra top
top_nick=comex_ncm.groupby('nick').KG_LIQUIDO.sum().sort_values().index[-2:]
top_uf=comex_ncm.groupby('SG_UF_NCM').KG_LIQUIDO.sum().sort_values().index[-2:]
top_pais=comex_ncm.groupby('CO_PAIS').KG_LIQUIDO.sum().sort_values().index[-2:]
top_urf=comex_ncm.groupby('CO_URF').KG_LIQUIDO.sum().sort_values().index[-2:]
comex_ncm=comex_ncm[(comex_ncm.nick.isin(top_nick))&(comex_ncm.SG_UF_NCM.isin(top_uf))&(comex_ncm.CO_PAIS.isin(top_pais))&(comex_ncm.CO_URF.isin(top_urf))]

# Salva datas de início e fim
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')
date_ini=comex_ncm.date.min()
date_fim=comex_ncm.date.max()

# Sparse Matrix 
# Matriz 1
anos=pd.DataFrame({'CO_ANO':np.arange(comex_ncm.CO_ANO.min(),comex_ncm.CO_ANO.max()+1),'key':0})
# Matriz 2
cat=comex_ncm[cat_cols].drop_duplicates().assign(key=0)
# Multiplica
c_c=anos.merge(cat,how='outer').drop(columns='key')        
comex_ncm=c_c.merge(comex_ncm,how='outer').fillna(0)
# Refaz data
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')
# Apara as bordas
comex_ncm=comex_ncm[(comex_ncm.date<=date_fim)&(comex_ncm.date>=date_ini)]

AttributeError: 'DataFrame' object has no attribute 'oc'

AttributeError: 'DataFrame' object has no attribute 'oc'

In [ ]:
# Salva copia para o loop
comex_ncm_t=comex_ncm.copy()

In [ ]:
test_pred=pd.DataFrame()
# while
a=1
while comex_ncm_t.date.max()-pd.DateOffset(years=l)+ pd.DateOffset(months=1)>=comex_ncm_t.date.min():
    date_ini=comex_ncm_t.date.max()-pd.DateOffset(years=l) + pd.DateOffset(months=1)
    
    # Finaliza arquivo para processamento
    comex_ncm_t2=comex_ncm_t[comex_ncm_t.date>=date_ini]
    test_pred_p=preve_comex_ncm(comex_ncm_t2,
                    target_cols=['oc','KG_LIQUIDO'],
                    cat_cols=cat_cols,
                    md_c=15,
                    md_r=1)
    #print('test_pred_p.oc.isna().sum()',test_pred_p.oc.isna().sum())
    test_pred=pd.concat([test_pred,test_pred_p])
    test_pred=test_pred.assign(while_=a)
    a+=1
    
    # prepara inicio de novo loop
    comex_ncm_t=comex_ncm_t[comex_ncm_t.date<comex_ncm_t.date.max()]

In [ ]:
test_pred.while_.unique()

In [ ]:
test_pred.to_csv('test_pred.csv',index=False)

In [ ]:
fim_time=time.time()
print('Tempo repet_pred: ',round((fim_time-ini_time)/60,2),' minutos.')